# Part Segmentation

In [1]:
# Import libraries
from pathlib import Path
import copy

import torch
from PIL import Image
import numpy as np

from training.networks import Generator
from torch_utils import misc

import dnnlib
import legacy

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
# Setup Constants
DEVICE = "cpu"
IMAGE_SIZE = 256
TRUNCATION = 0.7 # What is this?
NOISE_MODE = 'const'
NUM_SAMPLES = 3

IMSHOW_SIZE = 3 # What is this?
LATENT_DIMENSION = 256

PAPER256_CFG = dnnlib.EasyDict(ref_gpus=8,  kimg=25000,  mb=64, mbstd=8,  fmaps=0.5, lrate=0.0025, gamma=1, ema=20,  ramp=None, map=8)



In [3]:
# Setup Network Arguments
# What is c_dim? training_set.label_dim
# label_dim is label_shape
# We have no labels so c_dim is 0
COMMON_KWARGS = dict(c_dim=0, img_resolution=IMAGE_SIZE, img_channels=3)

GEN_KWARGS = dnnlib.EasyDict(class_name='training.networks.Generator', z_dim=512, w_dim=512, mapping_kwargs=dnnlib.EasyDict(), synthesis_kwargs=dnnlib.EasyDict())
DIS_KWARGS = dnnlib.EasyDict(class_name='training.networks.Discriminator', block_kwargs=dnnlib.EasyDict(), mapping_kwargs=dnnlib.EasyDict(), epilogue_kwargs=dnnlib.EasyDict())

GEN_KWARGS.synthesis_kwargs.channel_base = DIS_KWARGS.channel_base = int(PAPER256_CFG.fmaps * 32768)
GEN_KWARGS.synthesis_kwargs.channel_max = DIS_KWARGS.channel_max = 512
GEN_KWARGS.synthesis_kwargs.num_fp16_res = DIS_KWARGS.num_fp16_res = 4 # enable mixed-precision training

GEN_KWARGS.synthesis_kwargs.conv_clamp = DIS_KWARGS.conv_clamp = 256 # clamp activations to avoid float16 overflow
GEN_KWARGS.mapping_kwargs.num_layers = PAPER256_CFG.map

DIS_KWARGS.epilogue_kwargs.mbstd_group_size = PAPER256_CFG.mbstd

GEN_OPT_KWARGS = dnnlib.EasyDict(class_name='torch.optim.Adam', lr=PAPER256_CFG.lrate, betas=[0,0.99], eps=1e-8)
DIS_OPT_KWARGS = dnnlib.EasyDict(class_name='torch.optim.Adam', lr=PAPER256_CFG.lrate, betas=[0,0.99], eps=1e-8)


In [4]:
# Load Model
model_path = Path("./trained/network-snapshot-009676.pkl")

generator = dnnlib.util.construct_class_by_name(**GEN_KWARGS, **COMMON_KWARGS).train().requires_grad_(False).to(DEVICE) # subclass of torch.nn.Module
discriminator = dnnlib.util.construct_class_by_name(**DIS_KWARGS, **COMMON_KWARGS).train().requires_grad_(False).to(DEVICE) # subclass of torch.nn.Module
generator_copy = copy.deepcopy(generator).eval()

with dnnlib.util.open_url(str(model_path)) as f:
    resume_data = legacy.load_network_pkl(f)
    for name, module in [('G', generator), ('D', discriminator), ('G_ema', generator_copy)]:
        print("loading: ", name)
        misc.copy_params_and_buffers(resume_data[name], module, require_all=False)


loading:  G
loading:  D
loading:  G_ema


In [5]:
# Generate n Images
seeds = [i for i in range(10)]
label = torch.zeros([1, generator.c_dim], device=DEVICE)

for seed_idx, seed in enumerate(seeds):
    print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, generator.z_dim)).to(DEVICE)
    img = generator(z, label, truncation_psi=TRUNCATION, noise_mode=NOISE_MODE)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')


Generating image for seed 0 (0/10) ...


RuntimeError: "slow_conv_transpose2d_out_cpu" not implemented for 'Half'

In [ ]:
# Label Images

In [ ]:
# Train FewShotCNN

In [ ]:
# Load Test Images

In [ ]:
# Perform Part Segmentation